<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 81kB 3.3MB/s 
     |████████████████████████████████| 122kB 20.2MB/s 


In [5]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [6]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [7]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [8]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 8


In [9]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [10]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [11]:
landline_masterlist = landline.make_masterlist()

In [12]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210408 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210408 - 05 DOH Data Collect - Daily Report.csv


In [13]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [14]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [15]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [16]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [17]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [18]:
# quarantine facility daily patient census
quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [19]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [20]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [21]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [22]:
prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
suffix = ['_total', '_er', '_icu', '_ward'] 
triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
added = ['Augmen_total']

In [23]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [24]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [25]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,238361,2021-04-05,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,51,121,34,0,49,279,26,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,324725,2021-04-05,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,15,25,9,0,5,37,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,192867,2021-04-05,JUSTICE JOSE ABAD SANTOS GENERAL HOSPITAL,BINONDO,8,39,5,0,4,15,1,2,(02) 8254 2453,(02) 5310 1984,None,Government
3,117183,2021-04-05,QUIRINO MEMORIAL MEDICAL CENTER,QUEZON CITY,6,21,11,0,4,36,9,72,(02) 8913 4758,(02) 8421 2250,(02) 8913 4758,Government
4,165250,2021-04-05,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,5,2,2,6,6,11,14,(02) 8716 1853,None,(02) 8716 1853,Private
5,332260,2021-04-05,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,3,4,2,8,5,14,5,10,(02) 8723 0041,(02) 8893 5762,None,Private
6,36574,2021-04-05,EAST AVENUE MEDICAL CENTER,QUEZON CITY,3,39,22,0,57,167,44,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
7,187864,2021-04-05,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,5,10,4,1,3,43,(02) 8294 6711,None,(02) 8291 4259,Government
8,2808,2021-04-05,TONDO MEDICAL CENTER,TONDO I / II,2,18,4,0,16,12,2,96,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
9,174179,2021-04-05,RIZAL MEDICAL CENTER,CITY OF PASIG,2,26,19,0,16,89,3,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government


In [26]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,238362,2021-04-06,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,57,124,51,0,43,276,9,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,324726,2021-04-06,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,16,26,9,0,4,36,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,166650,2021-04-06,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,6,37,3,51,4,19,1,33,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
3,120262,2021-04-06,FATIMA UNIVERSITY MEDICAL CENTER CORPORATION,CITY OF VALENZUELA,5,13,1,2,2,11,2,10,None,(02) 8291 6538,(02) 8293 2705,Private
4,174180,2021-04-06,RIZAL MEDICAL CENTER,CITY OF PASIG,4,24,16,0,14,91,6,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
5,165251,2021-04-06,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,4,3,2,3,6,8,11,13,(02) 8716 1853,None,(02) 8716 1853,Private
6,332261,2021-04-06,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,4,4,2,7,4,14,5,12,(02) 8723 0041,(02) 8893 5762,None,Private
7,36575,2021-04-06,EAST AVENUE MEDICAL CENTER,QUEZON CITY,4,48,24,0,56,153,43,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
8,187865,2021-04-06,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,4,11,4,1,4,42,(02) 8294 6711,None,(02) 8291 4259,Government
9,57694,2021-04-06,UNIVERSITY OF PERPETUAL HELP DALTA MEDICAL CEN...,CITY OF LAS PIÑAS,3,3,6,0,11,33,9,0,(02) 8874 8515,(02) 8873 7210,(02) 8873 7210,Private


In [27]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,238363,2021-04-07,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,65,112,53,0,35,288,7,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,324727,2021-04-07,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,11,20,9,0,9,41,6,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
2,165252,2021-04-07,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,5,0,2,2,5,11,11,14,(02) 8716 1853,None,(02) 8716 1853,Private
3,166651,2021-04-07,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,5,36,2,48,5,20,2,36,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,332262,2021-04-07,DE LOS SANTOS MEDICAL CENTER,QUEZON CITY,4,6,2,7,4,12,5,12,(02) 8723 0041,(02) 8893 5762,None,Private
5,220503,2021-04-07,NATIONAL CHILDREN?S HOSPITAL,QUEZON CITY,4,20,5,0,9,57,8,0,(02) 8724 0656,None,(02) 8721 9139,Government
6,174181,2021-04-07,RIZAL MEDICAL CENTER,CITY OF PASIG,3,35,16,0,15,80,6,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
7,36576,2021-04-07,EAST AVENUE MEDICAL CENTER,QUEZON CITY,3,39,24,0,12,154,43,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
8,187866,2021-04-07,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,3,1,4,18,4,1,4,35,(02) 8294 6711,None,(02) 8291 4259,Government
9,180578,2021-04-07,AMANG RODRIGUEZ MEMORIAL MEDICAL CENTER,CITY OF MARIKINA,2,0,1,7,3,47,2,38,(02) 8942 0245,None,None,Government


In [28]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,27100,2021-04-07,FERNANDO AIR BASE HOSPITAL,LIPA CITY,10,10,1,4,10,10,0,0,+63 437781810,None,None,Government
1,59361,2021-04-07,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,17,10,12,1,13,1,11,+63 464190063,None,None,Government
2,119983,2021-04-07,"MARY MEDIATRIX MEDICAL CENTER, INC.",LIPA CITY,4,1,6,0,7,0,1,26,+63 437736800,+63 437736861,+63 437736861,Private
3,230003,2021-04-07,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,4,1,1,10,1,0,2,14,+63 495205626,None,None,Private
4,253970,2021-04-07,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,3,13,10,0,5,38,17,0,(02) 8988 3100,+63 464818000,None,Private
5,1973,2021-04-07,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,2,2,0,7,+63 434071103,None,+63 434071101,Private
6,144278,2021-04-07,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,2,0,3,0,2,4,0,0,+63 495341254,None,None,Private
7,308854,2021-04-07,LIPA MEDIX MEDICAL CENTER,LIPA CITY,2,4,6,4,2,4,0,4,+63 437563008,None,None,Private
8,237526,2021-04-07,"LOS BA?OS DOCTORS HOSPITAL AND MEDICAL CENTER,...",LOS BAÑOS,1,12,1,0,4,18,3,0,+63 495360100,None,+63 495361225,Private


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,154129,2021-04-07,JOSE B. LINGAD MEMORIAL REGIONAL HOSPITAL,CITY OF SAN FERNANDO (CAPITAL),23,0,15,29,59,0,13,216,+63 459613544,None,None,Government
1,249229,2021-04-07,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),13,17,23,4,50,66,25,8,+63 472372269,None,None,Government
2,25059,2021-04-07,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),5,63,50,24,25,30,1,76,+63 447910630,None,None,Government
3,70216,2021-04-07,"GRACE GENERAL HOSPITAL, INC.",CITY OF SAN JOSE DEL MONTE,2,3,1,0,2,11,0,0,+63 445194838,None,None,Private
4,282677,2021-04-07,SACRED HEART MEDICAL CENTER,ANGELES CITY,2,2,2,13,2,0,2,8,+63 456257876,None,None,Private
5,10060,2021-04-07,"SE?OR STO. NI?O HOSPITAL, INC.",CAMILING,1,0,4,0,3,7,0,4,+63 459340806,None,None,Private
6,164133,2021-04-07,"SACRED HEART HOSPITAL OF MALOLOS, INC.",CITY OF MALOLOS (CAPITAL),1,2,8,0,4,15,0,0,+63 447940561,None,+63 447912911,Private
7,195104,2021-04-07,"CENTRAL LUZON DOCTORS? HOSPITAL, INC.",CITY OF TARLAC (CAPITAL),1,35,5,0,7,16,0,0,+63 459820806,None,None,Private
8,204595,2021-04-07,WESLEYAN UNIVERSITY - PHILIPPINES,CABANATUAN CITY,1,29,1,0,1,0,0,0,+63 444634735,None,None,Private
9,268480,2021-04-07,ISAAC AND CATALINA MEDICAL CENTER,CITY OF BALANGA (CAPITAL),1,18,2,0,1,0,0,0,+63 472371435,None,None,Private


In [31]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100